In [2]:
# Databricks notebook source
# MAGIC %md This notebook is slightly modified version of `MLflow Training Tutorial` from [MLflow examples](https://github.com/mlflow/mlflow/tree/master/examples/sklearn_elasticnet_wine).
# MAGIC
# MAGIC It predicts the quality of wine using [sklearn.linear_model.ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html).
# MAGIC This is a base code and will be modified further during the `Databricks: Reproducible experiments with MLflow and Delta Lake` tutorial.
# MAGIC
# MAGIC Attribution
# MAGIC * The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# MAGIC * P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# MAGIC * Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

In [44]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestClassifier

import logging

from mlflow import log_metric, log_param, log_params, log_artifacts

In [52]:
# Wine Quality Sample
#def train(alpha=0.5, l1_ratio=0.5):
def train(max_depth=100, max_features=100, n_estimators=10):

    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    try:
        data = pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    with mlflow.start_run(experiment_id=experiment.experiment_id) as run:
        # Execute ElasticNet
        #lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr = RandomForestClassifier(max_depth=max_depth, max_features=max_features, n_estimators=n_estimators)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        
        # RF estimator
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("max_features", max_features)
        mlflow.log_param("n_estimators", n_estimators)
        #mlflow.log_param("alpha", alpha)
        #mlflow.log_param("l1_ratio", l1_ratio)
        
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        
        # Print out metrics
        #print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("RF model (max_depth=%f, max_features=%f, n_estimators=%f):" % (max_depth, max_features, n_estimators))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        

In [31]:
#mlflow.autolog()
mlflow.autolog(disable=True)

In [38]:
experiment = mlflow.get_experiment_by_name("MLOps-Lab9-RandomForest")
experiment

<Experiment: artifact_location='mlflow-artifacts:/622057648752947154', creation_time=1696436650659, experiment_id='622057648752947154', last_update_time=1696436650659, lifecycle_stage='active', name='MLOps-Lab9-RandomForest', tags={}>

In [53]:
train()

RF model (max_depth=100.000000, max_features=100.000000, n_estimators=10.000000):
  RMSE: 0.6557438524302001
  MAE: 0.36
  R2: 0.39074067514434496


In [55]:
for max_depth in range(1, 1001, 100): #alpha=0.5, l1_ratio=0.5
    for max_features in range(1, 1001, 100):
        train(max_depth, max_features)

RF model (max_depth=1.000000, max_features=1.000000, n_estimators=10.000000):
  RMSE: 0.7810249675906654
  MAE: 0.525
  R2: 0.13570188799546612
RF model (max_depth=1.000000, max_features=101.000000, n_estimators=10.000000):
  RMSE: 0.7984359711335656
  MAE: 0.5125
  R2: 0.09673762884772075
RF model (max_depth=1.000000, max_features=201.000000, n_estimators=10.000000):
  RMSE: 0.7984359711335656
  MAE: 0.5125
  R2: 0.09673762884772075
RF model (max_depth=1.000000, max_features=301.000000, n_estimators=10.000000):
  RMSE: 0.7984359711335656
  MAE: 0.5125
  R2: 0.09673762884772075
RF model (max_depth=1.000000, max_features=401.000000, n_estimators=10.000000):
  RMSE: 0.7984359711335656
  MAE: 0.5125
  R2: 0.09673762884772075
RF model (max_depth=1.000000, max_features=501.000000, n_estimators=10.000000):
  RMSE: 0.7984359711335656
  MAE: 0.5125
  R2: 0.09673762884772075
RF model (max_depth=1.000000, max_features=601.000000, n_estimators=10.000000):
  RMSE: 0.7984359711335656
  MAE: 0.5125
